# Drug Repurposing for Accelerated Therapeutic Discovery

## End-to-End Pipeline Using Graph Convolutional Networks

This notebook reproduces the complete pipeline:
1. Data Preprocessing
2. Graph Construction
3. GCN Model Training
4. Evaluation & Interpretation

**Dataset:** CTD (Comparative Toxicogenomics Database)  
**Model:** 2-Layer Graph Convolutional Network (GCN)  
**Task:** Drug-Disease Link Prediction


## Setup & Imports


In [ ]:
# Run the complete pipeline
import subprocess
import sys
from pathlib import Path

# Create output directories
Path("./outputs").mkdir(exist_ok=True)
Path("./outputs/plots").mkdir(exist_ok=True)

print("="*60)
print("Drug Repurposing Pipeline - Complete Execution")
print("="*60)

# Step 1: Preprocessing
print("\n[1/4] Running preprocessing...")
result = subprocess.run([sys.executable, "preprocess.py", 
                        "--top_chemicals", "150",
                        "--top_diseases", "100", 
                        "--top_genes", "200"],
                       capture_output=True, text=True)
if result.returncode == 0:
    print("✓ Preprocessing completed")
else:
    print("✗ Preprocessing failed:", result.stderr)

# Step 2: Graph Construction
print("\n[2/4] Building graph...")
result = subprocess.run([sys.executable, "build_graph.py"],
                       capture_output=True, text=True)
if result.returncode == 0:
    print("✓ Graph construction completed")
else:
    print("✗ Graph construction failed:", result.stderr)

# Step 3: Training (reduced epochs for notebook)
print("\n[3/4] Training GCN model...")
result = subprocess.run([sys.executable, "train_gcn.py",
                        "--epochs", "50"],  # Reduced for demo
                       capture_output=True, text=True)
if result.returncode == 0:
    print("✓ Training completed")
else:
    print("✗ Training failed:", result.stderr)

# Step 4: Evaluation
print("\n[4/4] Running evaluation...")
result = subprocess.run([sys.executable, "evaluate.py"],
                       capture_output=True, text=True)
if result.returncode == 0:
    print("✓ Evaluation completed")
else:
    print("✗ Evaluation failed:", result.stderr)

print("\n" + "="*60)
print("Pipeline execution complete!")
print("="*60)


## View Results


In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image, display

# Load metrics
with open("./outputs/test_metrics.json", "r") as f:
    metrics = json.load(f)

print("Test Set Performance:")
print(f"  ROC AUC:  {metrics['auc']:.4f}")
print(f"  AUPR:     {metrics['aupr']:.4f}")
if 'precision@10' in metrics:
    print(f"  Precision@10: {metrics['precision@10']:.4f}")


In [ ]:
# Display ROC and PR curves
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ROC Curve
roc_img = plt.imread("./outputs/plots/roc_curve.png")
axes[0].imshow(roc_img)
axes[0].axis('off')
axes[0].set_title("ROC Curve", fontsize=14)

# PR Curve
pr_img = plt.imread("./outputs/plots/pr_curve.png")
axes[1].imshow(pr_img)
axes[1].axis('off')
axes[1].set_title("Precision-Recall Curve", fontsize=14)

plt.tight_layout()
plt.show()


In [ ]:
# View top predictions
predictions_df = pd.read_csv("./outputs/predictions.csv")

# Show sample predictions for first disease
sample_disease = predictions_df['disease_id'].iloc[0]
sample_preds = predictions_df[predictions_df['disease_id'] == sample_disease].head(10)

print(f"\nTop 10 Predictions for Disease: {sample_disease}")
print(sample_preds[['chemical_id', 'score', 'rank', 'is_known']].to_string(index=False))


In [ ]:
# View interpretations
interp_df = pd.read_csv("./outputs/interpretation_top10.csv")

print("\nTop Prediction Interpretations:")
print(interp_df[['disease_id', 'chemical_id', 'score', 'intermediate_genes']].head(5).to_string(index=False))


## Summary

This notebook successfully executed the complete drug repurposing pipeline:

1. ✅ **Data Preprocessing**: Loaded CTD data, selected frequent nodes, created features
2. ✅ **Graph Construction**: Built homogeneous graph from heterogeneous interactions
3. ✅ **Model Training**: Trained 2-layer GCN for link prediction
4. ✅ **Evaluation**: Generated metrics, predictions, and interpretations

### Next Steps

- Launch the interactive demo: `streamlit run app.py`
- Explore detailed predictions in `./outputs/predictions.csv`
- View all output files in `./outputs/` directory

### Files Generated

- `model_best.pt` - Trained model weights
- `embeddings.npy` - Node embeddings  
- `predictions.csv` - Drug-disease predictions
- `test_metrics.json` - Performance metrics
- `interpretation_top10.csv` - Mechanistic interpretations
- `plots/` - Visualizations
